In [5]:
import torch
import torch.nn as nn
import snntorch as snn
import snntorch.functional as snnfunc
import snntorch.spikegen as snngen
from torch.utils.data import DataLoader, TensorDataset, random_split
import pandas as pd

In [6]:
class Net(nn.Module):
	def __init__(self, beta = 0.5, threshold = 0.5, membrane_zero = 0.3):
		super().__init__()
		self.fll = nn.Linear(4, 4, bias = False)
		self.fsl = snn.Leaky(beta = beta, threshold = threshold,
									learn_beta= True, 
									learn_threshold= True)
		
		self.sll = nn.Linear(4, 3, bias = False)
		self.ssl = snn.Leaky(beta = beta, threshold = threshold,
									learn_beta= True, 
									learn_threshold= True)
		
		
	def forward(self, spk_input):
		mem1 = self.fsl.init_leaky()
		mem2 = self.ssl.init_leaky()

		spk_output = []
		for step in range(spk_input.shape[1]):
			cur1 = self.fll(spk_input.to(torch.float32)[:, step])
			spk1, mem1 = self.fsl(cur1, mem1)
			cur2 = self.sll(spk1)
			spk2, mem2 = self.ssl(cur2, mem2)
			
			spk_output.append(spk2)
		
		return torch.stack(spk_output)

In [7]:
file_path = "../iris_folder/Iris.csv"
df = pd.read_csv(filepath_or_buffer= file_path, sep= ",", header= 0)
df.drop("Id", axis= 1, inplace= True)

# преобразование датафрейма
species = df["Species"].unique()

# При присваивании происходит изменение типа данных на object
for i in range(len(species)):
	df.loc[df["Species"] == species[i], "Species"] = i
df["Species"] = df["Species"].astype("int")

columns_headers = df.columns

# Не учитываю species
for header in columns_headers[:-1]:
	# Нормализую знеачения
	df[header] = df[header] / df[header].max()
# создание датасета для обучения
data = []
num_steps = 100

for header in columns_headers:
	data.append(torch.tensor(df[header].values))

# Транспонируем тензор, чтобы иметь features и target каждого образца
data = torch.stack(data, dim = 0).T

trains = snngen.rate(data= data[:, :-1], num_steps= num_steps)

# labels = snngen.targets_rate(data[:, -1], num_classes=3)
labels = data[:, -1]
print(trains.shape)
# Возможно полная хрень. Требует проверки
# Я уверен, что с осями какой-то косяк, так что придется переделывать при плохих результатах обучения
trains = trains.permute(1, 0, 2)
dataset = TensorDataset(trains, labels)
train_data, test_data = random_split(dataset, [0.8, 0.2])
train_data_loader = DataLoader(train_data, shuffle= True)
test_data_loader = DataLoader(test_data, shuffle= True)
lrng_rt = 5e-3

epochs = 5
net = Net()
optim = torch.optim.Adam(net.parameters(), lr = lrng_rt)

torch.Size([100, 150, 4])


In [15]:
for epoch in range(epochs):
		for trns, lbls in train_data_loader:
				optim.zero_grad()
				outputs = net(trns)
				
				loss_fn = snnfunc.loss.ce_count_loss()
				loss = loss_fn(spk_out=outputs, targets= lbls.to(torch.long))
				loss.backward()
				optim.step()

		print((f"Epoch {epoch}, Loss: {loss.item():.4f}"))

Epoch 0, Loss: 0.0000
Epoch 1, Loss: 0.0487
Epoch 2, Loss: 0.1270
Epoch 3, Loss: 0.0067
Epoch 4, Loss: 0.0070


In [17]:
# Testing accuracy
data = []
for trn, lbl in test_data_loader:
		prediction = net(trn)
		prediction = torch.mean(input= prediction.unsqueeze(0), dim = 1)

		data.append([prediction.argmax().item(), lbl.item()])
data = torch.tensor(data)
accurate = data[(data[:, 0] == data[:, 1]), 0].numel()
accuracy = accurate/data.shape[0]

print((f"accuracy : {accuracy:.2f}"))

accuracy : 0.97


In [19]:
for par, val in net.named_parameters():
    print(par, val)

torch.save(net.state_dict(), "fisher_iris_weights_leaky.pth")

fll.weight Parameter containing:
tensor([[ 0.5107,  0.2889, -0.5308, -0.6423],
        [ 0.0960, -0.0537,  0.3766,  0.0580],
        [-0.1288, -0.5120,  0.4117,  0.0970],
        [-0.3018, -0.7005, -0.2066, -0.2599]], requires_grad=True)
fsl.threshold Parameter containing:
tensor(0.4816, requires_grad=True)
fsl.beta Parameter containing:
tensor(0.3548, requires_grad=True)
sll.weight Parameter containing:
tensor([[ 0.7127, -0.1873, -0.7343, -0.3331],
        [ 0.3311,  0.1809,  0.0866,  0.2750],
        [-0.6120,  0.2930,  0.1601, -0.3958]], requires_grad=True)
ssl.threshold Parameter containing:
tensor(1.1666, requires_grad=True)
ssl.beta Parameter containing:
tensor(1.0154, requires_grad=True)
